In [1]:
import pickle
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
from utility.albumentations_helper import create_transform
from utility.others import clear_all
import time
from functions import DataGenerator,PredictGenerator,create_model,dot_distance,exp_loss,\
                        sigmoid_dot_distance,dot_sigmoid_distance,cross_entropy_loss,l2_distance,\
                        margin_loss_fun_factory,Xception_reduced,dot_distance_neg,\
                        TripletGenerator,create_model_triplet
from utility.grayscale_imagenet import Xception_greyscale
from utility.grayscale_DenseNet import DenseNet_greyscale
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
#from tensorflow.keras.applications import DenseNet121

In [2]:
color = False
shapes = (224,224,1)
HalfBatch = 8
margin = -2

### set up generator

In [3]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

In [4]:
aug = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1),Cutout(p=0.5)])
transform = create_transform(aug)  

aug_test = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform_test = create_transform(aug_test)    

gen_train = TripletGenerator(Ids_train,newWhale_train,transform,HalfBatch=HalfBatch)
gen_val = TripletGenerator(Ids_val,newWhale_val,transform_test,HalfBatch=HalfBatch)

### build model

In [5]:
conv_base = DenseNet_greyscale(121,(224,224,1),'max',False)
#conv_base = Xception_greyscale((256,256,1),'max',False)
# conv_base = Xception_reduced((256,256,1))

In [6]:
train_model,feature_model = create_model_triplet(1e-3,l2_distance,\
                                                 conv_base,color,margin)

### training

In [7]:
# conv_base = DenseNet_greyscale(121,(224,224,1),'max',False)
# train_model,feature_model = create_model2(1e-3,margin_loss_fun_factory(0.1,5),conv_base,color)
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

/home/will/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
713/713 [==============================] - 51s 71ms/step - loss: -1.2368 - val_loss: -1.2104
Epoch 2/5
713/713 [==============================] - 42s 58ms/step - loss: -1.1604 - val_loss: -0.9521
Epoch 3/5
713/713 [==============================] - 42s 59ms/step - loss: -1.1030 - val_loss: -1.0061
Epoch 4/5
713/713 [==============================] - 42s 59ms/step - loss: -1.0447 - val_loss: -0.9769
Epoch 5/5
713/713 [==============================] - 42s 59ms/step - loss: -1.0454 - val_loss: -0.8347
time:218.998108625412


Fine-Tune

In [8]:
feature_model.layers[0].trainable=True

In [9]:
train = False
for layer in feature_model.layers[0].layers:
    if 'conv5' in layer.name:
        train = True
    layer.trainable = train

In [12]:
train_model.compile(loss=None,optimizer=tf.keras.optimizers.Adam(lr=4e-4))

In [14]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

/home/will/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
713/713 [==============================] - 71s 100ms/step - loss: -1.1155 - val_loss: -1.3230
Epoch 2/5
713/713 [==============================] - 55s 77ms/step - loss: -1.3624 - val_loss: -1.4355
Epoch 3/5
713/713 [==============================] - 55s 78ms/step - loss: -1.5321 - val_loss: -1.4019
Epoch 4/5
713/713 [==============================] - 55s 78ms/step - loss: -1.5510 - val_loss: -1.5090
Epoch 5/5
713/713 [==============================] - 55s 78ms/step - loss: -1.6296 - val_loss: -1.4016
time:297.7761158943176


In [15]:
train_model.save('Models/DenseNet_Triplet1.h5')
feature_model.save('Models/feature_model_DenseNet_Triplet1.h5')

In [16]:
train = False
for layer in feature_model.layers[0].layers:
    if 'conv4' in layer.name:
        train = True
    layer.trainable = train

In [17]:
train_model.compile(loss=None,optimizer=tf.keras.optimizers.Adam(lr=1e-4))

In [19]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

/home/will/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
713/713 [==============================] - 106s 149ms/step - loss: -1.6694 - val_loss: -1.5632
Epoch 2/5
713/713 [==============================] - 79s 110ms/step - loss: -1.7263 - val_loss: -1.6662
Epoch 3/5
713/713 [==============================] - 79s 110ms/step - loss: -1.7704 - val_loss: -1.6344
Epoch 4/5
713/713 [==============================] - 79s 110ms/step - loss: -1.7852 - val_loss: -1.6852
Epoch 5/5
713/713 [==============================] - 78s 110ms/step - loss: -1.8173 - val_loss: -1.6928
time:433.1601552963257


In [20]:
train = True
for layer in feature_model.layers[0].layers:
    layer.trainable = train

In [22]:
train_model.compile(loss=None,optimizer=tf.keras.optimizers.Adam(lr=1e-5))

In [24]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

/home/will/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
713/713 [==============================] - 168s 236ms/step - loss: -1.8479 - val_loss: -1.7975
Epoch 2/5
713/713 [==============================] - 129s 180ms/step - loss: -1.8559 - val_loss: -1.8232
Epoch 3/5
713/713 [==============================] - 128s 180ms/step - loss: -1.8962 - val_loss: -1.8236
Epoch 4/5
713/713 [==============================] - 129s 180ms/step - loss: -1.8970 - val_loss: -1.7778
Epoch 5/5
713/713 [==============================] - 128s 180ms/step - loss: -1.8827 - val_loss: -1.8367
time:700.4021592140198


In [25]:
train_model.save('Models/DenseNet_Triplet2.h5')
feature_model.save('Models/feature_model_DenseNet_Triplet2.h5')